In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy import misc
import os
from data_utils import getDataSet, deprocessImage
import math

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test, classes_dic, mean = getDataSet('101_ObjectCategories/', 200, 300)

In [ ]:
print ('X_train shape: {}'.format(X_train.shape))
print ('y_train shape: {}'.format(y_train.shape))
print ('X_val shape: {}'.format(X_val.shape))
print ('y_val shape: {}'.format(y_val.shape))
print ('X_test shape: {}'.format(X_test.shape))
print ('y_test shape: {}'.format(y_test.shape))

In [ ]:
i = np.random.randint(0, 7145)
plt.imshow(np.uint8(deprocessImage(X_train[i], mean)))
print (classes_dic[y_train[i]])

In [ ]:
def convVGG(a, num_filters):
    return tf.layers.conv2d(a, filters=num_filters, kernel_size=(3, 3), strides=(1, 1), padding="same", kernel_initializer=tf.initializers.random_normal(stddev=0.01), bias_initializer=tf.initializers.zeros(), activation="relu", kernel_regularizer=tf.contrib.layers.l2_regularizer(5e-4))

In [ ]:
def FC(a, units):
    return tf.layers.dropout(tf.layers.dense(a, units=units, kernel_initializer=tf.initializers.random_normal(stddev=0.01), bias_initializer=tf.initializers.zeros(), activation="relu", kernel_regularizer=tf.contrib.layers.l2_regularizer(5e-4)))

In [ ]:
def VGG16Model(x, num_classes):
    a1 = convVGG(x, 64)
    a2 = convVGG(a1, 64)
    a3 = tf.layers.max_pooling2d(a2, pool_size=(2, 2), strides=(2, 2))
    
    a4 = convVGG(a3, 128)
    a5 = convVGG(a4, 128)
    a6 = tf.layers.max_pooling2d(a5, pool_size=(3, 3), strides=(2, 2))
    
    a7 = convVGG(a6, 256)
    a8 = convVGG(a7, 256)
    #a9 = convVGG(a8, 256)
    a10 = tf.layers.max_pooling2d(a8, pool_size=(3, 3), strides=(2, 2))
    
    a11 = convVGG(a10, 512)
    a12 = convVGG(a11, 512)
    #a13 = convVGG(a12, 512)
    a14 = tf.layers.max_pooling2d(a12, pool_size=(3, 3), strides=(2, 2))
    
    a15 = convVGG(a14, 512)
    a16 = convVGG(a15, 512)
    #a17 = convVGG(a16, 512)
    a18 = tf.layers.max_pooling2d(a16, pool_size=(3, 3), strides=(2, 2))
    
    a18 = tf.layers.flatten(a18)
    
    a19 = FC(a18, 4096)
    a20 = FC(a19, 4096)
    
    scores = tf.layers.dense(a20, units=num_classes)
    
    return scores

In [ ]:
def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=64, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    correct_prediction = tf.equal(tf.cast(tf.argmax(predict,1), tf.int32), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [mean_loss,correct_prediction,accuracy]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%Xd.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now }
            # get batch size
            actual_batch_size = yd[idx].shape[0]
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/float(actual_batch_size)))
            iter_cnt += 1
        total_correct = float(correct)/Xd.shape[0]
        total_loss = np.sum(losses, dtype=float)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct

In [ ]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 224, 224, 3])
y = tf.placeholder(tf.int32, [None])
is_training = tf.placeholder(tf.bool)

num_classes = len(classes_dic)

y_out = VGG16Model(X, num_classes)

total_loss = tf.losses.softmax_cross_entropy(tf.one_hot(y,num_classes),logits=y_out)
mean_loss = tf.reduce_mean(total_loss)

#global_step = tf.Variable(0, trainable=False)
#starter_learning_rate = 0.02
#learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
 #                                          500, 0.96, staircase=True)

optimizer = tf.train.MomentumOptimizer(0.01, 0.9)

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_step = optimizer.minimize(mean_loss)
    
    
saver = tf.train.Saver()

config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC'

sess = tf.Session(config = config)

sess.run(tf.global_variables_initializer())
train_accs = []
val_accs = []
max_val_acc = 0.0
for i in range(50):    
    print('Training')
    loss, acc = run_model(sess,y_out,mean_loss,X_train,y_train,1,40,100,train_step,True)
    train_accs.append(acc)
    print('Validation')
    loss, acc = run_model(sess,y_out,mean_loss,X_val,y_val,1,40)
    val_accs.append(acc)
    if acc > max_val_acc:
        max_val_acc = acc
        saver.save(sess, './model_tensorflow.ckpt')

In [ ]:
t = range(len(train_accs))
plt.plot(t, train_accs, 'r--', label = "train acc")
plt.plot(t, val_accs, 'b--', label = "test acc")
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.)
plt.savefig('train_val_acc.jpg')
plt.show()

In [ ]:
saver.restore(sess, "model_tensorflow.ckpt")

In [ ]:
print('Validation')
run_model(sess,y_out,mean_loss,X_val,y_val,1,40)

In [ ]:
print('Test')
run_model(sess,y_out,mean_loss,X_test,y_test,1,40)